#### Initialize Spark session

In [30]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("xor") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()

sc = spark.sparkContext


#### Read in data and merge dataset on ab_id

In [33]:
df = spark.read.option("inferSchema", "true").csv('pitches_preprocessed.csv', header = True)

In [34]:
df.count()

1712127

In [35]:
df.select('latent_next_pitch', 'latent_pitch_type').show()

+-----------------+-----------------+
|latent_next_pitch|latent_pitch_type|
+-----------------+-----------------+
|              0.0|              0.0|
|              0.0|              0.0|
|              7.0|              0.0|
|              7.0|              7.0|
|              0.0|              7.0|
|              6.0|              0.0|
|              6.0|              6.0|
|              7.0|              2.0|
|              7.0|              7.0|
|              3.0|              7.0|
|              0.0|              2.0|
|              2.0|              0.0|
|              6.0|              3.0|
|              6.0|              6.0|
|              6.0|              6.0|
|              6.0|              6.0|
|              0.0|              6.0|
|              2.0|              7.0|
|              0.0|              3.0|
|              0.0|              0.0|
+-----------------+-----------------+
only showing top 20 rows



In [36]:
df.dtypes

[('_c0', 'int'),
 ('outs', 'double'),
 ('pfx_x', 'double'),
 ('pfx_z', 'double'),
 ('pitch_num', 'double'),
 ('px', 'double'),
 ('pz', 'double'),
 ('start_speed', 'double'),
 ('sz_bot', 'double'),
 ('sz_top', 'double'),
 ('x0', 'double'),
 ('y0', 'double'),
 ('z0', 'double'),
 ('batter_id', 'int'),
 ('inning', 'int'),
 ('p_throws', 'int'),
 ('pitcher_id', 'int'),
 ('stand', 'int'),
 ('top', 'int'),
 ('score_difference', 'double'),
 ('latent_pitch_type', 'double'),
 ('latent_next_pitch', 'double'),
 ('count_status', 'int'),
 ('base_status', 'int'),
 ('binned_score_difference', 'int')]

In [37]:
df.select('binned_score_difference').distinct().rdd.map(lambda r: r[0]).collect()

[-1, 1, 3, -5, 5, 4, -4, -2, 2, -3, 0]

In [38]:
df = df.na.drop(subset=["count_status"])

In [39]:
df.groupby('binned_score_difference').count().show()

+-----------------------+------+
|binned_score_difference| count|
+-----------------------+------+
|                     -1|203766|
|                      1|206991|
|                      3| 97133|
|                     -5|116771|
|                      5|122646|
|                      4| 67897|
|                     -4| 63988|
|                     -2|139310|
|                      2|144543|
|                     -3| 91749|
|                      0|457333|
+-----------------------+------+



In [40]:
df.groupby('latent_pitch_type').count().show()

+-----------------+------+
|latent_pitch_type| count|
+-----------------+------+
|              8.0| 25268|
|              0.0|620753|
|              7.0| 41089|
|              1.0|269070|
|              4.0|145263|
|              3.0|172108|
|              2.0|198409|
|              6.0| 89226|
|              5.0|144449|
|              9.0|  6492|
+-----------------+------+



In [41]:
df.groupby('latent_next_pitch').count().show()

+-----------------+------+
|latent_next_pitch| count|
+-----------------+------+
|              8.0| 29974|
|              0.0|597164|
|              7.0| 39667|
|              1.0|283170|
|              4.0|138905|
|              3.0|193926|
|              2.0|194296|
|              6.0| 92221|
|              5.0|136358|
|              9.0|  6446|
+-----------------+------+



In [42]:
df.groupby('count_status').count().show()

+------------+------+
|count_status| count|
+------------+------+
|           1|233298|
|           6| 24340|
|           3|237259|
|           5|149360|
|           9| 50790|
|           4| 79288|
|           8|218893|
|           7|122587|
|          10|186938|
|          11|112276|
|           2|297098|
+------------+------+



In [43]:
df.columns

['_c0',
 'outs',
 'pfx_x',
 'pfx_z',
 'pitch_num',
 'px',
 'pz',
 'start_speed',
 'sz_bot',
 'sz_top',
 'x0',
 'y0',
 'z0',
 'batter_id',
 'inning',
 'p_throws',
 'pitcher_id',
 'stand',
 'top',
 'score_difference',
 'latent_pitch_type',
 'latent_next_pitch',
 'count_status',
 'base_status',
 'binned_score_difference']

In [44]:
df = df.select('outs', 'pfx_x', 'pfx_z', 'pitch_num', 'px', 'pz', 'start_speed', 'sz_bot', 'sz_top', 'x0', 'y0',
 'z0', 'inning', 'p_throws', 'stand', 'latent_pitch_type',
 'count_status','base_status', 'binned_score_difference','latent_next_pitch')

In [45]:
df.columns

['outs',
 'pfx_x',
 'pfx_z',
 'pitch_num',
 'px',
 'pz',
 'start_speed',
 'sz_bot',
 'sz_top',
 'x0',
 'y0',
 'z0',
 'inning',
 'p_throws',
 'stand',
 'latent_pitch_type',
 'count_status',
 'base_status',
 'binned_score_difference',
 'latent_next_pitch']

In [46]:
df = df.withColumn('binned_score_difference', df.binned_score_difference +5)

In [47]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(inputCols =["latent_pitch_type", "pitch_num", "base_status","binned_score_difference",
                                            "count_status"],
                                 outputCols =["latent_pitch_typeH", "pitch_numH", "base_statusH","binned_score_differenceH"
                                              ,"count_statusH"])
model = encoder.fit(df)
df = model.transform(df)

In [48]:
df.select('latent_pitch_typeH').take(5)

[Row(latent_pitch_typeH=SparseVector(9, {0: 1.0})),
 Row(latent_pitch_typeH=SparseVector(9, {0: 1.0})),
 Row(latent_pitch_typeH=SparseVector(9, {0: 1.0})),
 Row(latent_pitch_typeH=SparseVector(9, {7: 1.0})),
 Row(latent_pitch_typeH=SparseVector(9, {7: 1.0}))]

In [49]:
df.columns

['outs',
 'pfx_x',
 'pfx_z',
 'pitch_num',
 'px',
 'pz',
 'start_speed',
 'sz_bot',
 'sz_top',
 'x0',
 'y0',
 'z0',
 'inning',
 'p_throws',
 'stand',
 'latent_pitch_type',
 'count_status',
 'base_status',
 'binned_score_difference',
 'latent_next_pitch',
 'count_statusH',
 'base_statusH',
 'binned_score_differenceH',
 'latent_pitch_typeH',
 'pitch_numH']

In [50]:
df = df.select('outs','pfx_x','pfx_z','px','pz','start_speed','sz_bot','sz_top','x0',
               'y0','z0','inning','p_throws','stand','latent_pitch_typeH','pitch_numH',
               'base_statusH','binned_score_differenceH','count_statusH','latent_next_pitch')

In [51]:
df.select('latent_pitch_typeH','pitch_numH','base_statusH','binned_score_differenceH','count_statusH').take(4)

[Row(latent_pitch_typeH=SparseVector(9, {0: 1.0}), pitch_numH=SparseVector(14, {1: 1.0}), base_statusH=SparseVector(7, {0: 1.0}), binned_score_differenceH=SparseVector(10, {4: 1.0}), count_statusH=SparseVector(11, {2: 1.0})),
 Row(latent_pitch_typeH=SparseVector(9, {0: 1.0}), pitch_numH=SparseVector(14, {2: 1.0}), base_statusH=SparseVector(7, {0: 1.0}), binned_score_differenceH=SparseVector(10, {4: 1.0}), count_statusH=SparseVector(11, {5: 1.0})),
 Row(latent_pitch_typeH=SparseVector(9, {0: 1.0}), pitch_numH=SparseVector(14, {1: 1.0}), base_statusH=SparseVector(7, {0: 1.0}), binned_score_differenceH=SparseVector(10, {}), count_statusH=SparseVector(11, {2: 1.0})),
 Row(latent_pitch_typeH=SparseVector(9, {7: 1.0}), pitch_numH=SparseVector(14, {2: 1.0}), base_statusH=SparseVector(7, {0: 1.0}), binned_score_differenceH=SparseVector(10, {}), count_statusH=SparseVector(11, {3: 1.0}))]

In [52]:
def transData(data):
    return data.rdd.map(lambda r: [r[-1], Vectors.dense(r[:-6]), r[-2], r[-3], r[-4], r[-5], r[-6]]).\
           toDF(['label','features', 'count_statusH','binned_score_differenceH','base_statusH', 'pitch_numH',
                 'latent_pitch_typeH'])

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

data= transData(df)
data.show()

+-----+--------------------+---------------+------------------------+-------------+--------------+------------------+
|label|            features|  count_statusH|binned_score_differenceH| base_statusH|    pitch_numH|latent_pitch_typeH|
+-----+--------------------+---------------+------------------------+-------------+--------------+------------------+
|  0.0|[1.0,6.08,9.83,-0...| (11,[2],[1.0])|          (10,[4],[1.0])|(7,[0],[1.0])|(14,[1],[1.0])|     (9,[0],[1.0])|
|  0.0|[1.0,4.54,12.83,-...| (11,[5],[1.0])|          (10,[4],[1.0])|(7,[0],[1.0])|(14,[2],[1.0])|     (9,[0],[1.0])|
|  7.0|[0.0,-3.71,9.05,-...| (11,[2],[1.0])|              (10,[],[])|(7,[0],[1.0])|(14,[1],[1.0])|     (9,[0],[1.0])|
|  7.0|[0.0,4.87,-6.37,0...| (11,[3],[1.0])|              (10,[],[])|(7,[0],[1.0])|(14,[2],[1.0])|     (9,[7],[1.0])|
|  0.0|[0.0,1.64,-4.12,0...| (11,[8],[1.0])|              (10,[],[])|(7,[0],[1.0])|(14,[3],[1.0])|     (9,[7],[1.0])|
|  6.0|[0.0,-2.47,9.54,-...|(11,[10],[1.0])|            

In [53]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

norm = Normalizer(inputCol='features', outputCol='features_norm', p=1.0)
data = norm.transform(data)

In [54]:
data.select('features_norm').take(1)

[Row(features_norm=DenseVector([0.0056, 0.034, 0.055, -0.003, 0.0151, 0.5239, 0.0104, 0.0214, 0.0121, 0.2796, 0.0344, 0.0056, 0.0, 0.0]))]

In [55]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['features_norm', 'latent_pitch_typeH','pitch_numH','base_statusH',
                                         'binned_score_differenceH','count_statusH'], outputCol = 'features_fin')

data = assembler.transform(data)

In [56]:
data = data.select('label', 'features_fin')
data.select('features_fin').take(1)

[Row(features_fin=SparseVector(65, {0: 0.0056, 1: 0.034, 2: 0.055, 3: -0.003, 4: 0.0151, 5: 0.5239, 6: 0.0104, 7: 0.0214, 8: 0.0121, 9: 0.2796, 10: 0.0344, 11: 0.0056, 14: 1.0, 24: 1.0, 37: 1.0, 48: 1.0, 56: 1.0}))]

#### Split data into training and test data

In [57]:
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

train = data.filter(data['label'] == 9.0).sample(1165/8285)\
.union(data.filter(data['label'] == 8.0).sample(1165/30932)).union(data.filter(data['label'] == 7.0).sample(1165/50315))\
.union(data.filter(data['label'] == 6.0).sample(1165/109587)).union(data.filter(data['label'] == 5.0).sample(1165/177058))\
.union(data.filter(data['label'] == 4.0).sample(1165/178452)).union(data.filter(data['label'] == 3.0).sample(1165/211755))\
.union(data.filter(data['label'] == 2.0).sample(1165/244465)).union(data.filter(data['label'] == 1.0).sample(1165/330376))\
.union(data.filter(data['label'] == 0.0).sample(1165/763599))

In [58]:
df.columns

['outs',
 'pfx_x',
 'pfx_z',
 'px',
 'pz',
 'start_speed',
 'sz_bot',
 'sz_top',
 'x0',
 'y0',
 'z0',
 'inning',
 'p_throws',
 'stand',
 'latent_pitch_typeH',
 'pitch_numH',
 'base_statusH',
 'binned_score_differenceH',
 'count_statusH',
 'latent_next_pitch']

#### specify layers for the neural network: input layer of size 11 (features), two intermediate of size 5 and 4 and output of size 7 (classes)

# Currently not working so experimenting with other code
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

layers = [11, 5, 4, 4, 3 , 11]

# create the trainer and set its parameters
FNN = MultilayerPerceptronClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",\
                                         maxIter=100, layers=layers, blockSize=128, seed=1234)
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
# Chain indexers and forest in a Pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, FNN, labelConverter])
# train the model
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)


In [73]:
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

layers = [65, 10, 5, 10]

trainer = MultilayerPerceptronClassifier(maxIter=1000, featuresCol = 'features_fin',
                                         layers=layers, blockSize=128, seed=1234)

#### Train the model

In [60]:
model = trainer.fit(train)

#### Compute accuracy on the test set

In [61]:
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.4239524407592986


With layers = [23, 20, 15, 6, 11] accuracy = 0.36312009234584086

With layers = [23, 20, 15, 11] accuracy = 0.3631949095780308

With layers = [22, 30, 15, 11] accuracy = 0.3613826699538746

With layers = [68, 10, 5, 11] accuracy = 0.42275469889813827

With layers = [65, 10, 5, 10] accuracy = 0.4239524407592986

In [158]:
evaluator = MulticlassClassificationEvaluator(metricName="weightedPrecision")
evaluator.evaluate(predictionAndLabels)

0.40100858600575917

In [159]:
evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
evaluator.evaluate(predictionAndLabels)

0.42275469889813827

In [62]:
pred = ['outs', 'pfx_x', 'pfx_z', 'pitch_num', 'px', 'pz', 'start_speed',
        'sz_bot', 'sz_top', 'x0', 'y0','z0', 'inning', 'p_throws', 'stand']

In [65]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
from pyspark.ml.feature import StandardScaler
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

assembler = VectorAssembler(
                            inputCols=[c for c in df.columns if c in pred],
                            outputCol='features').setHandleInvalid('skip')
output = assembler.transform(df)
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)
scaleroutput = scaler.fit(output)
scaledoutput = scaleroutput.transform(output)

In [66]:
pca = PCA(k=5, inputCol="scaledFeatures", outputCol="pcaFeatures")
PC = pca.fit(scaledoutput)

In [69]:
scaledoutput = PC.transform(scaledoutput)

In [70]:
scaledoutput.take(1)

[Row(outs=1.0, pfx_x=6.08, pfx_z=9.83, px=-0.532, pz=2.702, start_speed=93.7, sz_bot=1.86, sz_top=3.83, x0=2.161, y0=50.0, z0=6.151, inning=1, p_throws=0, stand=0, latent_pitch_typeH=SparseVector(9, {0: 1.0}), pitch_numH=SparseVector(14, {1: 1.0}), base_statusH=SparseVector(7, {0: 1.0}), binned_score_differenceH=SparseVector(10, {4: 1.0}), count_statusH=SparseVector(11, {2: 1.0}), latent_next_pitch=0.0, features=DenseVector([1.0, 6.08, 9.83, -0.532, 2.702, 93.7, 1.86, 3.83, 2.161, 50.0, 6.151, 1.0, 0.0, 0.0]), scaledFeatures=DenseVector([0.0226, 1.1516, 0.9073, -0.5945, 0.4672, 0.8873, 1.8885, 1.7978, 1.6659, 0.0, 0.7366, -1.4967, -1.652, -1.176]), pcaFeatures=DenseVector([-1.7682, -2.8534, -1.4608, -1.8148, -0.4139]))]

In [76]:
layers = [5, 10, 5, 10]

trainer = MultilayerPerceptronClassifier(maxIter=1000, featuresCol = 'pcaFeatures', labelCol = 'latent_next_pitch',
                                         layers =layers, blockSize=128, seed=1234)

In [78]:
splits = scaledoutput.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

In [ ]:
model = trainer.fit(train)

In [ ]:
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))